In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import glob
import pandas as pd
import os
import glob

# Personnal Import 
from utils.utilities import get_mode_date2path
from build_inputs.load_raw_data import load_subway_15_min,load_CRITER,load_netmob

# Data
- Validation individuelles, aggrégée **3min**
- Metro (entrée/sortie)  **15 min**
- Sensor, aggrégé **6 min**

### Notes et questionnements
- Un 'VAL_ARRET_CODE' peut être l'arrêt de plusieurs mêmes bus, voir d'un même bus et d'un même arrêt de métro. Où d'un même bus et d'un même arrêt de tram. Je dois donc nommer différement les VAL_ARRET_CODE de chacun des modes. Une proposition est de mettre le mode (B,S,T) devant les id. Comme ça on pourra regrouper sans soucis.
- La moyenne des déplacement de la df_subway est de 5 trajet toute les 3 minutes, quelque soit la station et l'heure (d'ouverture) considéré. Max 88.
- Ok on a aggrégé 3 min, mais est-ce qu'on peut recouper les sorties 3min avec les validation + Sortie de métro 15 min? 

In [2]:
# Subway 15 Min :                                   [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,~, , ,~,~, , ],[J,F,M, , , , , , , , , ]
# Validation Individuelles  Nov 2019 - Mai 2021     [ , , , , , , , , , ,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M, , , , , , , ]
# NetMob 15 Min :                                   [ , ,M,A,M, , , , , , , ],[ , , , , , , , , , , , ],[ , , , , , , , , , , , ]
# Sensor 6 Min :  Janvier 2019 - Decembre 2020      [J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,J,J,A,S,O,N,D],[J,F,M,A,M,J,J,A,S,O,N,D]

## Load Validation Individuelles
Load 3 df : df_sub, df_tram, df_bus

In [3]:
FOLDER_PATH = '../../../data/rrochas/raw_data/keolis_data_2019-2020/'
valid_ind_path = f"{FOLDER_PATH}Sub_Tram_11_2019_03_2020"
dates = ['11-2019','12-2019','1-2020','2-2020','3-2020']

subway_paths, tramway_paths, bus_paths = sorted(glob.glob(os.path.join(valid_ind_path, "*df_subway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_tramway*.csv"))),sorted(glob.glob(os.path.join(valid_ind_path, "*df_bus*.csv")))
mode_month2path = get_mode_date2path([subway_paths,tramway_paths,bus_paths],['sub','tram','bus'])

for name in ['sub','tram','bus']:
    globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])

/tmp/ipykernel_108571/2947046094.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/tmp/ipykernel_108571/2947046094.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/tmp/ipykernel_108571/2947046094.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/tmp/ipykernel_108571/2947046094.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'df_{name}'] = pd.concat([pd.read_csv(mode_month2path[name][d],index_col = 0) for d in dates])
/tmp/ipykern

In [9]:
from datetime import datetime 
from load_inputs.subway_in import get_trigram_correspondance

start,end = datetime(2019,11,1),datetime(2020,4,1)
freqs = ['2min','3min','5min','6min','10min','15min','30min','1H']
freqs.reverse()

save_path = '../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle'

for name in ['sub','tram','bus']:
    df_i = globals()[f'df_{name}']
    df_i.VAL_DATE = pd.to_datetime(df_i.VAL_DATE)
    for freq in freqs:
        reindex = pd.date_range(start,end,freq = freq)
        if name == 'sub':
            df = df_i[['Flow','VAL_DATE','CRS_SENS_TRAJET','LIB_STA_SIFO']].groupby(['LIB_STA_SIFO',pd.Grouper(key='VAL_DATE',freq=freq)]).agg(Flow = ('Flow','sum'))
            df = df.reset_index()
            df = df.pivot(index = 'VAL_DATE',columns='LIB_STA_SIFO',values ='Flow').fillna(0)
            df_corres = get_trigram_correspondance()
            df.rename(columns = {row.INDIV:row.COD_TRG for k,row in df_corres.iterrows()})
            df.to_csv(f"{save_path}/subway_indiv_{freq}/subway_indiv_{freq}.csv")
        elif name == 'tram':
            raise NotImplementedError(f"Mode {name} not implemented yet")
        elif name == 'bus':
            raise NotImplementedError(f"Mode {name} not implemented yet")
if False:
    save_path = '../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle'
    for temp_agg in ['2min','3min','5min','10min','15min','30min','1H']:
        df_emitted = df[['id_sortie','date_sortie','id_retour']].groupby(['id_sortie',pd.Grouper(freq=temp_agg,key='date_sortie')]).agg(volume = ('id_retour','count'))
        df_attracted = df[['id_sortie','date_retour','id_retour']].groupby(['id_retour',pd.Grouper(freq=temp_agg,key='date_retour')]).agg(volume = ('id_sortie','count'))
        df_emitted.to_csv(f"{save_path}/velov_emitted_by_station{temp_agg}.csv")
        df_attracted.to_csv(f"{save_path}/velov_attracted_by_station{temp_agg}.csv")

/tmp/ipykernel_108571/3290188844.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  reindex = pd.date_range(start,end,freq = freq)
/tmp/ipykernel_108571/3290188844.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df_i[['Flow','VAL_DATE','CRS_SENS_TRAJET','LIB_STA_SIFO']].groupby(['LIB_STA_SIFO',pd.Grouper(key='VAL_DATE',freq=freq)]).agg(Flow = ('Flow','sum'))


OSError: Cannot save file into a non-existent directory: '../../../../data/rrochas/prediction_validation/agg_data/validation_individuelle/subway_indiv_1H'

In [ ]:
df = pd.read_csv()



LIB_STA_SIFO,AMP,BEL,BRO,CHA,COR,CPA,CRO,CUI,CUS,DEB,...,PER,GUI,JAU,REP,SAN,SAX,GER,VMY,SOI,JEA
VAL_DATE,,,,,,,,,,,,,,,,,,,,,
2019-11-01 00:00:00,26.0,250.0,21.0,89.0,133.0,10.0,34.0,1.0,8.0,24.0,...,108.0,20.0,23.0,18.0,65.0,103.0,10.0,42.0,4.0,97.0
2019-11-01 04:00:00,2.0,60.0,9.0,34.0,14.0,0.0,10.0,3.0,27.0,14.0,...,68.0,5.0,15.0,4.0,16.0,15.0,25.0,10.0,27.0,20.0
2019-11-01 05:00:00,17.0,174.0,34.0,175.0,54.0,12.0,35.0,22.0,40.0,72.0,...,260.0,29.0,85.0,41.0,43.0,128.0,134.0,107.0,77.0,53.0
2019-11-01 06:00:00,24.0,142.0,31.0,205.0,91.0,7.0,47.0,28.0,50.0,83.0,...,203.0,25.0,91.0,84.0,50.0,149.0,129.0,102.0,158.0,68.0
2019-11-01 07:00:00,32.0,150.0,39.0,267.0,76.0,6.0,107.0,35.0,76.0,71.0,...,242.0,26.0,95.0,72.0,93.0,187.0,56.0,122.0,166.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-30 19:00:00,17.0,79.0,21.0,122.0,28.0,3.0,32.0,16.0,23.0,27.0,...,69.0,30.0,37.0,24.0,28.0,88.0,15.0,47.0,57.0,20.0
2020-03-30 20:00:00,16.0,73.0,12.0,88.0,26.0,3.0,32.0,9.0,17.0,20.0,...,52.0,16.0,28.0,23.0,15.0,48.0,3.0,39.0,46.0,7.0
2020-03-30 21:00:00,4.0,33.0,6.0,63.0,11.0,2.0,11.0,7.0,7.0,8.0,...,31.0,8.0,10.0,4.0,7.0,34.0,3.0,21.0,34.0,7.0


In [14]:
df

### Exemple de Selection d'une période, avec aggregation temporelle, et sauvegarde

In [6]:
from datetime import datetime 

time_step_per_hour = 4
start,end = datetime(2019,11,1),datetime(2020,4,1)

freq = f'{60/time_step_per_hour}min'
reindex = pd.date_range(start,end,freq = freq)

df_sub.VAL_DATE = pd.to_datetime(df_sub.VAL_DATE)
df = df_sub[['Flow','VAL_DATE','CRS_SENS_TRAJET','LIB_STA_SIFO']].groupby(['LIB_STA_SIFO',pd.Grouper(key='VAL_DATE',freq=freq)]).agg(Flow = ('Flow','sum'))
df = df.reset_index()
df  =df.pivot(index = 'VAL_DATE',columns='LIB_STA_SIFO',values ='Flow').fillna(0)

df = df.reindex(reindex,fill_value=0)
#df.to_csv(f'data/subway{freq}_from_validation_individuelles_{start.month}_{start.year}_{end.month}_{end.year}.csv')
df

LIB_STA_SIFO,AMPERE,BELLECOUR,BROTTEAUX,CHARPENNES,CORDELIERS,CROIX PAQUET,CROIX ROUSSE,CUIRE,CUSSET,DEBOURG,...,PERRACHE,PLACE GUICHARD,PLACE JEAN JAURES,REPUBLIQUE,SANS SOUCI,SAXE GAMBETTA,STADE DE GERLAND,VALMY,VAULX-EN-VELIN LA SOIE,VIEUX LYON
2019-11-01 00:00:00,20.0,164.0,10.0,59.0,88.0,5.0,20.0,1.0,5.0,21.0,...,89.0,13.0,16.0,12.0,49.0,68.0,8.0,23.0,3.0,59.0
2019-11-01 00:15:00,6.0,82.0,11.0,30.0,43.0,3.0,8.0,0.0,2.0,3.0,...,19.0,7.0,7.0,3.0,11.0,33.0,2.0,16.0,0.0,38.0
2019-11-01 00:30:00,0.0,4.0,0.0,0.0,2.0,2.0,6.0,0.0,1.0,0.0,...,0.0,0.0,0.0,3.0,5.0,2.0,0.0,3.0,1.0,0.0
2019-11-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-11-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-31 23:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-31 23:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-31 23:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Load Subway 15 min

In [15]:
txt_path = "Métro 15 minutes 2019 2020.txt"

df_metro_funi_2019_2020 = load_subway_15_min(f"{FOLDER_PATH}{txt_path}")
df_metro_funi_2019_2020.head()

,datetime,Station,Code ligne,in,out
0,2019-01-01 00:00:00,Ampère Victor Hugo,A,2,4.0
1,2019-01-01 00:15:00,Ampère Victor Hugo,A,3,2.0
2,2019-01-01 00:30:00,Ampère Victor Hugo,A,3,7.0
3,2019-01-01 00:45:00,Ampère Victor Hugo,A,1,9.0
4,2019-01-01 01:00:00,Ampère Victor Hugo,A,0,0.0


## Load Velo'V 

In [ ]:
import os 
import sys 
import pandas as pd
import chardet
import json 
from shapely.geometry import Point
import geopandas as gpd


def get_velov_data(year,quarter,folder_path):
    ''' Return shared bike dataframe from "Lyon Velov"  
    'year' in [2019,2020]
    'quarter' in ['T1','T2','T3','T4']
    '''
    # usefull to get the "encoding" of the csv, so that "pd.read_csv" can work well
    pathway = f'{folder_path}/Trajets VELOV {year}/VELOV_TRAJETS_{str(year)}_{quarter}.csv'
    with open(pathway, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(100000))      
    # ...
    
    df_velov = pd.read_csv(pathway, encoding = result['encoding'], sep = ";")
    df_velov.columns=['id_sortie','nom_sortie','date_sortie','id_retour','nom_retour','date_retour']
    df_velov.date_sortie = pd.to_datetime(df_velov.date_sortie)
    df_velov.date_retour = pd.to_datetime(df_velov.date_retour)
    return(df_velov)



FOLDER_PATH = '../../../data/rrochas/raw_data/VELO-LYON'
df = pd.concat([get_velov_data(year,Ti,FOLDER_PATH) for year in [2019,2020] for Ti in ['T1','T2','T3','T4']]) 
print('length df: ',len(df))
df.head()


length df:  15586007


,id_sortie,nom_sortie,date_sortie,id_retour,nom_retour,date_retour
0,6005,6005 - PLACE EDGAR QUINET,2019-01-01 00:00:00,2024.0,2024 - RÉPUBLIQUE / MAUPIN,2019-01-01 00:20:00
1,10072,10072 - JACQUES BREL,2019-01-01 00:01:00,4012.0,4012 - PLACE ADRIEN GODIEN,2019-01-01 00:09:00
2,6037,6037 - CITÉ INTERNATIONALE / INTERPOL,2019-01-01 00:02:00,10025.0,10025 - TOTEM,2019-01-01 00:32:00
3,3012,3012 - PLACE DU CHÂTEAU,2019-01-01 00:06:00,7002.0,7002 - UNIVERSITÉS LYON III / LYON II,2019-01-01 00:26:00
4,1002,1002 - OPÉRA,2019-01-01 00:07:00,7056.0,7056 - PLACE RASPAIL,2019-01-01 00:22:00


In [ ]:
html_save_path = 'velov_map.html'
json_path = 'locations/pvo_patrimoine_voirie.pvostationvelov.json'
stations_json = json.load(open(f"{FOLDER_PATH}/{json_path}",'rb'))

gdf = gpd.GeoDataFrame(
    [
        {**feat['properties'],
         'geometry': Point(*feat['geometry']['coordinates'])}
        for feat in stations_json['features']
    ],
    crs="EPSG:4326"
)
gdf.head()

In [ ]:
# save:
if False:
    save_path = '../../../../data/rrochas/prediction_validation/agg_data/velov'
    for temp_agg in ['2min','3min','5min','10min','15min','30min','1H']:
        df_emitted = df[['id_sortie','date_sortie','id_retour']].groupby(['id_sortie',pd.Grouper(freq=temp_agg,key='date_sortie')]).agg(volume = ('id_retour','count'))
        df_attracted = df[['id_sortie','date_retour','id_retour']].groupby(['id_retour',pd.Grouper(freq=temp_agg,key='date_retour')]).agg(volume = ('id_sortie','count'))
        df_emitted.to_csv(f"{save_path}/velov_emitted_by_station{temp_agg}.csv")
        df_attracted.to_csv(f"{save_path}/velov_attracted_by_station{temp_agg}.csv")

## Load NetMob 15 min

In [ ]:
FOLDER_PATH = '../../Data/NetMob/'
apps = [app for app in os.listdir(FOLDER_PATH) if ((app != 'Lyon.geojson') and (not app.startswith('.'))) ]   # Avoid hidden folder and Lyon.geojson

In [5]:
app = apps[0]
folder_days = [day for day in os.listdir(f'{FOLDER_PATH}/{app}') if (not day.startswith('.')) ] 
day = folder_days[0]
txt_paths = sorted(glob.glob(os.path.join(f'{FOLDER_PATH}/{app}/{day}', "*.txt")))
txt_paths

['../../Data/NetMob//Apple_Video/20190504/Lyon_Apple_Video_20190504_DL.txt',
 '../../Data/NetMob//Apple_Video/20190504/Lyon_Apple_Video_20190504_UL.txt']

In [6]:
txt_path = txt_paths[0]
df_Apple_Vid_DL = load_netmob(txt_path,day)
df_Apple_Vid_DL.head()

,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,02:15,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
tile_id,,,,,,,,,,,,,,,,,,,,,
66,1834,1596,1378,1575,637,421,203,1366,547,254,...,4114,2992,2887,3297,4634,4208,4886,2095,2066,919
353,2000,1687,1357,1548,764,484,195,2492,591,407,...,4673,3288,3156,3778,5123,4400,5300,2187,2176,984
354,1621,1540,1258,1381,490,375,94,1131,453,302,...,3887,3128,3048,3337,4582,3792,4519,2189,2309,984
640,1803,1493,1201,1332,632,514,73,2951,428,370,...,4404,3352,3082,3512,4697,4056,4862,2155,2156,1074
641,1621,1583,1296,1383,493,415,75,1087,402,368,...,4064,3426,3206,3517,4834,4011,4719,2256,2230,1073


## Load Sensor 6 min

In [ ]:
years = [2019,2020]
#FOLDER_PATH = '../../Data/Comptages_Velo_Routier/CRITER/'
FOLDER_PATH = '../../../data/rrochas/raw_data/Comptages_Velo_Routier/CRITER/'
list_files = sorted(glob.glob(os.path.join(f'{FOLDER_PATH}6 min {years[0]}', "*.txt"))) + sorted(glob.glob(os.path.join(f'{FOLDER_PATH}6 min {years[1]}', "*.txt")))
oct_2019_mar_2020 = list_files[9:15]

In [ ]:
df = load_CRITER(oct_2019_mar_2020[0])
len_init = len(df)
print((list_files[0]))
df.head()

/home/rrochas/prediction-validation/build_inputs/load_raw_data.py:48: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(txt_path, sep=';',infer_datetime_format=True)


../../../data/rrochas/raw_data/Comptages_Velo_Routier/CRITER/6 min 2019/6mn_01_Janvier_2019.txt


,ID_POINT_MESURE,NOM_POINT_MESURE,DEBIT_HEURE,TAUX_HEURE,HORODATE,NOMBRE_ECH_1_MIN_MANQUANTS,day,str_hour_min,hour_min
0,1,Berthelot_Servant_Mace,380,3,2019-10-01 00:00:00,0,1,0:0,0.0
1,1,Berthelot_Servant_Mace,240,2,2019-10-01 00:06:00,0,1,0:6,1.0
2,1,Berthelot_Servant_Mace,270,2,2019-10-01 00:12:00,0,1,0:12,2.0
3,1,Berthelot_Servant_Mace,270,2,2019-10-01 00:18:00,0,1,0:18,3.0
4,1,Berthelot_Servant_Mace,200,1,2019-10-01 00:24:00,0,1,0:24,4.0


# 1er Etape : Prédiction Métro
- On va d'abord prédire la demande sur une ligne (disons A).  
- On va comparer des modèle : LSTM, CNN, CNN-LSTM, GNN.
    - A priori pas de "raison" que le GNN marche mieux. Si c'est le cas, c'est peut être simplement que le modèle est plus complexe, mais j'ai du mal à croire que si on donne les bonnes informations (historique -7d, -1d, -4,3,2,1t), on a des meilleurs résultats avec GNN. Sauf si il y a des relation asynchrone "récurrentes", mais sans causalité. De la même manière que l'historique -7d sert de référence, mais ne témoigne pas d'un lien causal.
- Identifier des moments interessants : anomalies sur entrée/sortie métro. Voir les prédictions sur ces moments là particulier.
